### In this Study, there are two seperate files.One of them is Main and other is Sales file.
### Here the problem is, if one customer orders a product,it involves in Sales file,but not in Main File as Out of Stock.
### So I will try to find(predict) Out of Stocks' in Main File due to analyses and Modelling.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from datetime import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tabulate import tabulate
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import ClusterCentroids
import imblearn

In [3]:
#read main file
df_main = pd.read_excel('main_file.xlsx')

In [4]:
df_main.head()

,Prod_ID,Date,Normal_Selling_Rate,Qty_In_Stock,Discount_Rate
0,1345504,2018-09-15,9.62,13,0.00000
1,1165247,2018-09-15,7.92,8,-0.05303
2,1209027,2018-09-15,2.82,18,0.00000
3,141469,2018-09-15,32.50,16,0.00000
4,196467,2018-09-15,8.39,0,0.00000


In [5]:
df_main['Date'] =  pd.to_datetime(df_main['Date'],format='%Y-%m-%d')
df_main.set_index('Date', inplace=True)

In [6]:
df_main.head()

,Prod_ID,Normal_Selling_Rate,Qty_In_Stock,Discount_Rate
Date,,,,
2018-09-15,1345504,9.62,13,0.00000
2018-09-15,1165247,7.92,8,-0.05303
2018-09-15,1209027,2.82,18,0.00000
2018-09-15,141469,32.50,16,0.00000
2018-09-15,196467,8.39,0,0.00000


In [7]:
len(df_main)

625700

### total row size is 625700

In [8]:
len(df_main.Prod_ID.unique())

22754

### number of distinct products is 22754

In [9]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 625700 entries, 2018-09-15 to 2018-10-15
Data columns (total 4 columns):
Prod_ID                625700 non-null int64
Normal_Selling_Rate    625700 non-null float64
Qty_In_Stock           625700 non-null int64
Discount_Rate          625700 non-null float64
dtypes: float64(2), int64(2)
memory usage: 23.9 MB


### there is 31 days of time based information

In [11]:
df_main.isnull().sum()

Prod_ID                0
Normal_Selling_Rate    0
Qty_In_Stock           0
Discount_Rate          0
dtype: int64

In [12]:
#read sales file
df_sales = pd.read_excel('sales.xlsx')

In [13]:
df_sales.head()

,Order_Date,Deliver_Date,Ordered_Quant,OoS_Quant,Prod_ID
0,2018-09-15 11:44:00.000000,2018-09-15 17:12:00.000000,2,2,1321889
1,2018-09-15 11:44:00.000000,2018-09-15 17:12:00.000000,1,0,1266892
2,2018-09-15 11:44:00.000000,2018-09-15 17:12:00.000000,0,0,1300321
3,2018-09-15 11:44:00.000000,2018-09-15 17:12:00.000000,1,0,139970
4,2018-09-15 11:44:00.000000,2018-09-15 17:12:00.000000,2,2,1266765


In [14]:
df_sales['Order_Date'] =  pd.to_datetime(df_sales['Order_Date'])
df_sales['Deliver_Date'] =  pd.to_datetime(df_sales['Deliver_Date'])

In [15]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11753 entries, 0 to 11752
Data columns (total 5 columns):
Order_Date       11753 non-null datetime64[ns]
Deliver_Date     11753 non-null datetime64[ns]
Ordered_Quant    11753 non-null int64
OoS_Quant        11753 non-null int64
Prod_ID          11753 non-null int64
dtypes: datetime64[ns](2), int64(3)
memory usage: 459.2 KB


In [16]:
df_sales.head()

,Order_Date,Deliver_Date,Ordered_Quant,OoS_Quant,Prod_ID
0,2018-09-15 11:44:00,2018-09-15 17:12:00,2,2,1321889
1,2018-09-15 11:44:00,2018-09-15 17:12:00,1,0,1266892
2,2018-09-15 11:44:00,2018-09-15 17:12:00,0,0,1300321
3,2018-09-15 11:44:00,2018-09-15 17:12:00,1,0,139970
4,2018-09-15 11:44:00,2018-09-15 17:12:00,2,2,1266765


In [17]:
df_sales['Order_Date'].dt.date.unique()

array([datetime.date(2018, 9, 15), datetime.date(2018, 9, 16),
       datetime.date(2018, 9, 17), datetime.date(2018, 9, 18),
       datetime.date(2018, 9, 19), datetime.date(2018, 9, 20),
       datetime.date(2018, 9, 21), datetime.date(2018, 9, 22),
       datetime.date(2018, 9, 23), datetime.date(2018, 9, 24),
       datetime.date(2018, 9, 25), datetime.date(2018, 9, 26),
       datetime.date(2018, 9, 27), datetime.date(2018, 9, 28),
       datetime.date(2018, 9, 29), datetime.date(2018, 9, 30),
       datetime.date(2018, 10, 1), datetime.date(2018, 10, 2),
       datetime.date(2018, 10, 3), datetime.date(2018, 10, 4),
       datetime.date(2018, 10, 5), datetime.date(2018, 10, 6),
       datetime.date(2018, 10, 7), datetime.date(2018, 10, 8),
       datetime.date(2018, 10, 9), datetime.date(2018, 10, 10),
       datetime.date(2018, 10, 11), datetime.date(2018, 10, 12),
       datetime.date(2018, 10, 13), datetime.date(2018, 10, 14),
       datetime.date(2018, 10, 15), datetime.date(

In [18]:
df_sales['Deliver_Date'].dt.date.unique()

array([datetime.date(2018, 9, 15), datetime.date(2018, 9, 16),
       datetime.date(2018, 9, 17), datetime.date(2018, 9, 18),
       datetime.date(2018, 9, 22), datetime.date(2018, 9, 19),
       datetime.date(2018, 9, 20), datetime.date(2018, 9, 21),
       datetime.date(2018, 9, 23), datetime.date(2018, 9, 24),
       datetime.date(2018, 9, 25), datetime.date(2018, 9, 26),
       datetime.date(2018, 9, 27), datetime.date(2018, 9, 28),
       datetime.date(2018, 9, 29), datetime.date(2018, 9, 30),
       datetime.date(2018, 10, 1), datetime.date(2018, 10, 2),
       datetime.date(2018, 10, 3), datetime.date(2018, 10, 4),
       datetime.date(2018, 10, 6), datetime.date(2018, 10, 5),
       datetime.date(2018, 10, 12), datetime.date(2018, 10, 7),
       datetime.date(2018, 10, 9), datetime.date(2018, 10, 8),
       datetime.date(2018, 10, 11), datetime.date(2018, 10, 10),
       datetime.date(2018, 10, 13), datetime.date(2018, 10, 14),
       datetime.date(2018, 10, 17), datetime.date(

In [19]:
df_sales.isnull().sum()

Order_Date       0
Deliver_Date     0
Ordered_Quant    0
OoS_Quant        0
Prod_ID          0
dtype: int64

In [20]:
df_main.head()

,Prod_ID,Normal_Selling_Rate,Qty_In_Stock,Discount_Rate
Date,,,,
2018-09-15,1345504,9.62,13,0.00000
2018-09-15,1165247,7.92,8,-0.05303
2018-09-15,1209027,2.82,18,0.00000
2018-09-15,141469,32.50,16,0.00000
2018-09-15,196467,8.39,0,0.00000


### lets find products which normal_selling_rate is bigger than Qty_In_Stock this calculation may be useful in further works below and at the same time these values may indicate anomalies.

In [22]:
def is_diff(x):
    if x['Qty_In_Stock'] - x['Normal_Selling_Rate']<0:
        val = 1
    else:
        val = 0
    return val

In [23]:
df_main['is_diffx'] = df_main.apply(is_diff, axis=1)

In [24]:
df_main.head()

,Prod_ID,Normal_Selling_Rate,Qty_In_Stock,Discount_Rate,is_diffx
Date,,,,,
2018-09-15,1345504,9.62,13,0.00000,0
2018-09-15,1165247,7.92,8,-0.05303,0
2018-09-15,1209027,2.82,18,0.00000,0
2018-09-15,141469,32.50,16,0.00000,1
2018-09-15,196467,8.39,0,0.00000,1


### So some of products arent in stock even they have some selling records

### lets check them

In [25]:
df_main_diff = df_main[df_main.is_diffx == 1]
diffx_products=df_main_diff.Prod_ID.unique().tolist()

In [26]:
len(diffx_products)

17068

In [27]:
diff_rate=(17068/22754)*100
print(diff_rate)

75.01098707919486


### %75 of products have a differenced values at their normal selling attitudes apart from stock situation 

In [28]:
df_main[['is_diffx','Prod_ID']].groupby(['Prod_ID']).sum()['is_diffx'].sort_values(ascending=False).head(25)

Prod_ID
111826     31
1217855    31
1217889    31
1217888    31
1217887    31
1217885    31
1217883    31
1217877    31
1217876    31
1217875    31
1217872    31
1217871    31
1217867    31
110109     31
1217864    31
1217863    31
1217860    31
1217890    31
1217891    31
1217892    31
1217909    31
110072     31
1217914    31
1217913    31
1217912    31
Name: is_diffx, dtype: int64

### Some products are Out of Stock for all 31 days

In [32]:
#lets look at one example
df_main_1134 = df_main[df_main.Prod_ID == 1134]

df_main_1134

,Prod_ID,Normal_Selling_Rate,Qty_In_Stock,Discount_Rate,is_diffx
Date,,,,,
2018-09-15,1134,9.33,0,0.0,1
2018-09-16,1134,9.33,0,0.0,1
2018-09-17,1134,9.33,0,0.0,1
2018-09-18,1134,9.33,0,0.0,1
2018-09-19,1134,9.33,0,0.0,1
2018-09-20,1134,9.33,0,0.0,1
2018-09-21,1134,9.33,0,0.0,1
2018-09-22,1134,9.33,0,0.0,1
2018-09-23,1134,9.33,0,0.0,1


In [33]:
#lets check it in also sales file
df_sales_1134 = df_sales[df_sales.Prod_ID == 1134]

df_sales_1134

,Order_Date,Deliver_Date,Ordered_Quant,OoS_Quant,Prod_ID


### So we can think the Main data as general stock information about products which holds general(normal) selling and the stock quantities.

### Lets find the real anomalies which is not in the stock in the main file but lies in sales data because we cannot decide truely, what is their behavior in sales data.

In [34]:
df_sales['just_order_date'] = df_sales['Order_Date'].dt.date

In [35]:
df_sales['just_order_date'] =  pd.to_datetime(df_sales['just_order_date'],format='%Y-%m-%d')

In [36]:
df_sales.head()

,Order_Date,Deliver_Date,Ordered_Quant,OoS_Quant,Prod_ID,just_order_date
0,2018-09-15 11:44:00,2018-09-15 17:12:00,2,2,1321889,2018-09-15
1,2018-09-15 11:44:00,2018-09-15 17:12:00,1,0,1266892,2018-09-15
2,2018-09-15 11:44:00,2018-09-15 17:12:00,0,0,1300321,2018-09-15
3,2018-09-15 11:44:00,2018-09-15 17:12:00,1,0,139970,2018-09-15
4,2018-09-15 11:44:00,2018-09-15 17:12:00,2,2,1266765,2018-09-15


In [37]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11753 entries, 0 to 11752
Data columns (total 6 columns):
Order_Date         11753 non-null datetime64[ns]
Deliver_Date       11753 non-null datetime64[ns]
Ordered_Quant      11753 non-null int64
OoS_Quant          11753 non-null int64
Prod_ID            11753 non-null int64
just_order_date    11753 non-null datetime64[ns]
dtypes: datetime64[ns](3), int64(3)
memory usage: 551.0 KB


### merge two data to find real anomalies.

In [49]:
df_unioned=pd.merge(df_sales, df_main,  how='left', left_on=['just_order_date','Prod_ID'], right_on = ['Date','Prod_ID'])

In [50]:
df_unioned.head(10)

,Order_Date,Deliver_Date,Ordered_Quant,OoS_Quant,Prod_ID,just_order_date,Normal_Selling_Rate,Qty_In_Stock,Discount_Rate,is_diffx
0,2018-09-15 11:44:00,2018-09-15 17:12:00,2,2,1321889,2018-09-15,14.85,2.0,0.000000,1.0
1,2018-09-15 11:44:00,2018-09-15 17:12:00,1,0,1266892,2018-09-15,21.31,9.0,0.000000,1.0
2,2018-09-15 11:44:00,2018-09-15 17:12:00,0,0,1300321,2018-09-15,13.72,86.0,-0.278426,0.0
3,2018-09-15 11:44:00,2018-09-15 17:12:00,1,0,139970,2018-09-15,11.31,54.0,-0.076039,0.0
4,2018-09-15 11:44:00,2018-09-15 17:12:00,2,2,1266765,2018-09-15,14.05,1.0,0.000000,1.0
5,2018-09-15 11:44:00,2018-09-15 17:12:00,6,6,1247688,2018-09-15,10.28,4.0,0.000000,1.0
6,2018-09-15 11:44:00,2018-09-15 17:12:00,2,0,1132619,2018-09-15,9.33,14.0,0.000000,0.0
7,2018-09-15 11:44:00,2018-09-15 17:12:00,1,1,1132617,2018-09-15,17.60,11.0,0.000000,1.0
8,2018-09-15 11:44:00,2018-09-15 17:12:00,2,0,1202126,2018-09-15,NaN,NaN,NaN,NaN
9,2018-09-15 11:44:00,2018-09-15 17:12:00,1,0,18995,2018-09-15,6.74,14.0,0.000000,0.0


In [51]:
len(df_unioned)

11753

In [52]:
len(df_sales.Prod_ID.unique().tolist())

3507

### lets define Out Of Stocks based on unioned data.
### Calculation will be done based on if a product's Qty_In_Stock in any day, is bigger than Out Of Stock Quantity.They will NOT labeled as Out of Stock and others will be labeled as Out of Stock.

In [53]:
def is_OOS(x):
    if x['Qty_In_Stock'] - x['OoS_Quant']>0:
        val = 0
    else:
        val = 1
    return val

In [54]:
df_unioned['is_OoS'] = df_unioned.apply(is_OOS, axis=1)

In [55]:
df_unioned.head(5)

,Order_Date,Deliver_Date,Ordered_Quant,OoS_Quant,Prod_ID,just_order_date,Normal_Selling_Rate,Qty_In_Stock,Discount_Rate,is_diffx,is_OoS
0,2018-09-15 11:44:00,2018-09-15 17:12:00,2,2,1321889,2018-09-15,14.85,2.0,0.000000,1.0,1
1,2018-09-15 11:44:00,2018-09-15 17:12:00,1,0,1266892,2018-09-15,21.31,9.0,0.000000,1.0,0
2,2018-09-15 11:44:00,2018-09-15 17:12:00,0,0,1300321,2018-09-15,13.72,86.0,-0.278426,0.0,0
3,2018-09-15 11:44:00,2018-09-15 17:12:00,1,0,139970,2018-09-15,11.31,54.0,-0.076039,0.0,0
4,2018-09-15 11:44:00,2018-09-15 17:12:00,2,2,1266765,2018-09-15,14.05,1.0,0.000000,1.0,1


### So all sales data labeled regard to OoS calculation.

In [56]:
len(df_unioned)

11753

### Prepare new data for prediction

In [57]:
df_for_pred=df_unioned[['is_OoS','Normal_Selling_Rate','Qty_In_Stock','Discount_Rate','is_diffx','Prod_ID']]

In [58]:
df_for_pred.head()

,is_OoS,Normal_Selling_Rate,Qty_In_Stock,Discount_Rate,is_diffx,Prod_ID
0,1,14.85,2.0,0.000000,1.0,1321889
1,0,21.31,9.0,0.000000,1.0,1266892
2,0,13.72,86.0,-0.278426,0.0,1300321
3,0,11.31,54.0,-0.076039,0.0,139970
4,1,14.05,1.0,0.000000,1.0,1266765


In [59]:
df_for_pred[['is_OoS']].groupby(['is_OoS']).size()

is_OoS
0    10145
1     1608
dtype: int64

In [60]:
df_for_pred.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11753 entries, 0 to 11752
Data columns (total 6 columns):
is_OoS                 11753 non-null int64
Normal_Selling_Rate    10481 non-null float64
Qty_In_Stock           10481 non-null float64
Discount_Rate          10481 non-null float64
is_diffx               10481 non-null float64
Prod_ID                11753 non-null int64
dtypes: float64(4), int64(2)
memory usage: 642.7 KB


### Check nulls

In [61]:
df_for_pred.isnull().sum()

is_OoS                    0
Normal_Selling_Rate    1272
Qty_In_Stock           1272
Discount_Rate          1272
is_diffx               1272
Prod_ID                   0
dtype: int64

In [62]:
df_null=df_for_pred.columns[df_for_pred.isnull().any()].tolist()
df_null

['Normal_Selling_Rate', 'Qty_In_Stock', 'Discount_Rate', 'is_diffx']

In [64]:
for i in df_null:
    y=df_for_pred.loc[df_for_pred[i].isnull()]
    null_products=y.Prod_ID.unique().tolist()

len(null_products)

371

### 371 distinct products has nulls.

In [65]:
dfx= df_for_pred[df_for_pred.Prod_ID.isin(null_products)]
len(dfx)

3339

### These 371 products has 3339 rows in unioned dataframe.

In [66]:
dfx.is_OoS.sum()

1346

### and OoS rate in these products which contain nulls 1346, so it is the %83 of all OoS rates !

In [67]:
#lets check it in also sales file
df_unioned_1202126 = df_unioned[df_unioned.Prod_ID == 1202126]

df_unioned_1202126


,Order_Date,Deliver_Date,Ordered_Quant,OoS_Quant,Prod_ID,just_order_date,Normal_Selling_Rate,Qty_In_Stock,Discount_Rate,is_diffx,is_OoS
8,2018-09-15 11:44:00.000000,2018-09-15 17:12:00.000000,2,0,1202126,2018-09-15,NaN,NaN,NaN,NaN,1
980,2018-09-17 15:52:00.000000,2018-09-17 17:35:00.000000,1,0,1202126,2018-09-17,NaN,NaN,NaN,NaN,1
1600,2018-09-19 11:01:00.000000,2018-09-19 12:17:00.000000,1,0,1202126,2018-09-19,NaN,NaN,NaN,NaN,1
1716,2018-09-19 13:20:00.000000,2018-09-19 14:33:00.000000,1,0,1202126,2018-09-19,NaN,NaN,NaN,NaN,1
1902,2018-09-19 17:38:00.000000,2018-09-19 20:28:00.000000,1,0,1202126,2018-09-19,NaN,NaN,NaN,NaN,1
2135,2018-09-20 14:25:00.000000,2018-09-20 15:44:00.000000,1,0,1202126,2018-09-20,NaN,NaN,NaN,NaN,1
2247,2018-09-20 22:04:00.000000,2018-09-21 09:44:00.000000,1,0,1202126,2018-09-20,NaN,NaN,NaN,NaN,1
2884,2018-09-22 14:06:00.000000,2018-09-23 10:08:00.000000,1,0,1202126,2018-09-22,NaN,NaN,NaN,NaN,1
3190,2018-09-23 02:47:00.000000,2018-09-23 13:10:00.000000,1,0,1202126,2018-09-23,NaN,NaN,NaN,NaN,1
3581,2018-09-24 17:40:00.000000,2018-09-25 20:24:00.000000,1,0,1202126,2018-09-24,NaN,NaN,NaN,NaN,1


### But here is the problem is the null values is related to time based occurences.
### In backward or forward manner,the null values can be predicted using time series models.
### But it will lead only to reverse current is_OoS values.
### Instead of it, lets discard null values and try to model them.

In [68]:
#get the same data without nulls by inner join
df_unioned2=pd.merge(df_sales, df_main,  how='inner', left_on=['just_order_date','Prod_ID'], right_on = ['Date','Prod_ID'])

In [69]:
df_unioned2.head()

,Order_Date,Deliver_Date,Ordered_Quant,OoS_Quant,Prod_ID,just_order_date,Normal_Selling_Rate,Qty_In_Stock,Discount_Rate,is_diffx
0,2018-09-15 11:44:00,2018-09-15 17:12:00,2,2,1321889,2018-09-15,14.85,2,0.000000,1
1,2018-09-15 11:44:00,2018-09-15 17:12:00,1,0,1266892,2018-09-15,21.31,9,0.000000,1
2,2018-09-15 11:44:00,2018-09-15 17:12:00,0,0,1300321,2018-09-15,13.72,86,-0.278426,0
3,2018-09-15 11:44:00,2018-09-15 17:12:00,1,0,139970,2018-09-15,11.31,54,-0.076039,0
4,2018-09-15 11:44:00,2018-09-15 17:12:00,2,2,1266765,2018-09-15,14.05,1,0.000000,1


In [70]:
df_unioned2['is_OoS'] = df_unioned2.apply(is_OOS, axis=1)

In [71]:
df_unioned2[['is_OoS']].groupby(['is_OoS']).size()

is_OoS
0    10145
1      336
dtype: int64

## OoS rates are so low now.

In [72]:
df_for_pred2=df_unioned2[['is_OoS','Normal_Selling_Rate','Qty_In_Stock','Discount_Rate','is_diffx']]

In [73]:
df_for_pred2.head()

,is_OoS,Normal_Selling_Rate,Qty_In_Stock,Discount_Rate,is_diffx
0,1,14.85,2,0.000000,1
1,0,21.31,9,0.000000,1
2,0,13.72,86,-0.278426,0
3,0,11.31,54,-0.076039,0
4,1,14.05,1,0.000000,1


In [74]:
df_for_pred2.isnull().sum()

is_OoS                 0
Normal_Selling_Rate    0
Qty_In_Stock           0
Discount_Rate          0
is_diffx               0
dtype: int64

In [76]:
df_for_pred2.iloc[:,0] = df_for_pred2.loc[:,"is_OoS"].astype("category")

In [77]:
df_for_pred2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10481 entries, 0 to 10480
Data columns (total 5 columns):
is_OoS                 10481 non-null category
Normal_Selling_Rate    10481 non-null float64
Qty_In_Stock           10481 non-null int64
Discount_Rate          10481 non-null float64
is_diffx               10481 non-null int64
dtypes: category(1), float64(2), int64(2)
memory usage: 419.7 KB


### Because of low OoS rates, I will use SMOTE algorithm to make data balanced.

In [78]:
X = df_for_pred2.iloc[:,1:]
y = df_for_pred2.iloc[:,0]

In [80]:
os = SMOTE(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
columns = X_train.columns
os_data_X,os_data_y=os.fit_sample(X_train, y_train)
os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
os_data_y= pd.DataFrame(data=os_data_y,columns=['is_OoS'])
# we can Check the numbers of our data
print("length of oversampled data is ",len(os_data_X))
print("Number of In Stock in oversampled data",len(os_data_y[os_data_y['is_OoS']==0]))
print("Number of Out Of Stock",len(os_data_y[os_data_y['is_OoS']==1]))
print("Proportion of In Stock data in oversampled data is ",len(os_data_y[os_data_y['is_OoS']==0])/len(os_data_X))
print("Proportion of Out of Stock data in oversampled data is ",len(os_data_y[os_data_y['is_OoS']==1])/len(os_data_X))

length of oversampled data is  14174
Number of In Stock in oversampled data 7087
Number of Out Of Stock 7087
Proportion of In Stock data in oversampled data is  0.5
Proportion of Out of Stock data in oversampled data is  0.5


In [81]:
X=os_data_X
y=os_data_y['is_OoS']

# now perform logistic regression

In [83]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

C:\Users\Hincal\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [84]:
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.98


In [85]:
confusion_matrixx = confusion_matrix(y_test, y_pred)
print(confusion_matrixx)

[[2033   79]
 [  15 2126]]


In [86]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.96      0.98      2112
           1       0.96      0.99      0.98      2141

   micro avg       0.98      0.98      0.98      4253
   macro avg       0.98      0.98      0.98      4253
weighted avg       0.98      0.98      0.98      4253



# lets try also random forest

In [87]:
clf = RandomForestClassifier(n_estimators=20, max_depth=5,
                             random_state=0)
clf.fit(X_train, y_train)  
y_pred2 = clf.predict(X_test)

In [88]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred2))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred2))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred2))) 

Mean Absolute Error: 0.01598871384904773
Mean Squared Error: 0.01598871384904773
Root Mean Squared Error: 0.12644648610794895


In [89]:
print(confusion_matrix(y_test,y_pred2))  
print(classification_report(y_test,y_pred2))  
print(accuracy_score(y_test, y_pred2))  
print(" Prediciton: ",accuracy_score(y_pred2,y_test))

[[2068   44]
 [  24 2117]]
              precision    recall  f1-score   support

           0       0.99      0.98      0.98      2112
           1       0.98      0.99      0.98      2141

   micro avg       0.98      0.98      0.98      4253
   macro avg       0.98      0.98      0.98      4253
weighted avg       0.98      0.98      0.98      4253

0.9840112861509522
 Prediciton:  0.9840112861509522


### As seen from logistic Regression and Random Forest models, the results are good. Of course there are a lot of things different can be done but for this time it seems enough to predict unseen products on main data.

### LETS TURN TO MAIN DATA WHICH WANTED TO BE PREDICTED AS OoS WITHOUT PRODUCTS WHICH ARE ALREADY IN SALES DATA.

In [90]:
in_sales_products=df_unioned2.Prod_ID.unique().tolist()

In [91]:
df_other= df_main[~df_main.Prod_ID.isin(in_sales_products)]

In [92]:
df_other.head()

,Prod_ID,Normal_Selling_Rate,Qty_In_Stock,Discount_Rate,is_diffx
Date,,,,,
2018-09-15,1165247,7.92,8,-0.053030,0
2018-09-15,141469,32.50,16,0.000000,1
2018-09-15,196467,8.39,0,0.000000,1
2018-09-15,196060,2.21,0,0.000000,1
2018-09-15,1228001,11.31,34,-0.173298,0


In [93]:
len(df_other)

524779

In [94]:
len(df_other.Prod_ID.unique())

19415

In [95]:
df_other_for_prediction=df_other.reset_index()[['Normal_Selling_Rate','Qty_In_Stock','Discount_Rate','is_diffx']]

In [96]:
df_other_for_prediction.head()

,Normal_Selling_Rate,Qty_In_Stock,Discount_Rate,is_diffx
0,7.92,8,-0.053030,0
1,32.50,16,0.000000,1
2,8.39,0,0.000000,1
3,2.21,0,0.000000,1
4,11.31,34,-0.173298,0


In [97]:
y_pred_other = clf.predict(df_other_for_prediction)

In [98]:
df_other_for_prediction['IS_OoS']=y_pred_other

In [99]:
df_other_for_prediction.head()

,Normal_Selling_Rate,Qty_In_Stock,Discount_Rate,is_diffx,IS_OoS
0,7.92,8,-0.053030,0,0
1,32.50,16,0.000000,1,0
2,8.39,0,0.000000,1,1
3,2.21,0,0.000000,1,1
4,11.31,34,-0.173298,0,0


### It is finished and Labeled the Products as OoS in Main File by making Predictions.

# HINCAL TOPCUOGLU

# THANKS